In [58]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata

URL = "https://www.ndr.de/fernsehen/barrierefreie_angebote/leichte_sprache/Alle-Nachrichten-in-Leichter-Sprache,leichtesprachearchiv110.html"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
articles_single = []
articles_collection = []

for i in range(1, 240):
    url = "https://www.ndr.de/fernsehen/barrierefreie_angebote/leichte_sprache/leichtesprachearchiv110_page-%s.html" % (i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    print(url)
    headlines = soup.find_all("h2")
    for headline in headlines:
        #print(headline.prettify(), end="\n"*2)
        links = headline.find_all("a")  
        for link in links:
            if "Mehr-Nachrichten-vom" in link["href"]:
                articles_collection.append(link["href"])
            else:
                articles_single.append(link["href"])

In [ ]:
df = pd.DataFrame(columns=['URL', 'Titel', 'Text_Leicht'])
i = 1

for article in articles_collection:
    url = 'https://www.ndr.de%s' % (article)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    end_of_article = False
    for heading in soup.find_all("h2"):  # find separators, in this case h2 nodes
        article_text = ''
        for sibling in heading.find_next_siblings():
            if sibling.name == "h2":  # iterate through siblings until separator is encoutnered
                break
            t = sibling.getText()
            if "Diese Nachrichten sind vom" in t:
                end_of_article = True
                break
            else:
                clean_text = unicodedata.normalize("NFKD",t)
                if "-------------------------------------------------------------------" not in clean_text:
                    article_text = article_text + (' '.join(clean_text.split()))
        
        df_article = pd.DataFrame({
            'URL': url,
            'Titel': heading.text,
            'Text_Leicht': article_text},
            index=[i]
        )

        i += 1
        df = pd.concat([df, df_article])

        if end_of_article:
            break
    print(url)

In [79]:
df.to_csv("ndr_article_collection.csv")

In [ ]:
df = pd.DataFrame(columns=['URL', 'Titel', 'Text_Leicht'])
i = 1
for article in articles_single:
    url = 'https://www.ndr.de%s' % (article)

    if url != 'https://www.ndr.de/fernsehen/barrierefreie_angebote/index.html':
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")

        article_title = ''
        article_text = ''
        for heading in soup.find_all("h1"):
            article_title = unicodedata.normalize("NFKD",heading.getText()).replace('\n', '')
            break

        for p in soup.find_all("p"):
            clean_text = unicodedata.normalize("NFKD",p.getText())
            if "Diese Nachricht ist vom " in clean_text:
                break
            article_text = article_text + ' '+ (' '.join(clean_text.split()))

        df_article = pd.DataFrame({
            'URL': url,
            'Titel': article_title,
            'Text_Leicht': article_text},
            index=[i]
        )

        i += 1
        df = pd.concat([df, df_article])

        print(url)

   

In [107]:
df.to_csv("ndr_article_single.csv")

In [56]:
import unicodedata
blocks = {}
end_of_article = False
for heading in soup.find_all("h2"):  # find separators, in this case h2 nodes
    values = []
    for sibling in heading.find_next_siblings():
        if sibling.name == "h2":  # iterate through siblings until separator is encoutnered
            break
        t = sibling.getText()
        if "Diese Nachrichten sind vom" in t:
            end_of_article = True
            break
        else:
            clean_text = unicodedata.normalize("NFKD",t)
            if "-------------------------------------------------------------------" not in clean_text:
                values.append(' '.join(clean_text.split()))
    blocks[heading.text] = values
    if end_of_article:
        break

        
print(blocks)

{'Niedersachsen: Autobahn 7 gesperrt': ['Am Donnerstag hat die Polizei einen Teil von der A7 gesperrt.', 'A7 ist die Abkürzung für Autobahn 7.', 'Dieser Teil ist in der Nähe von Salzgitter.', 'Salzgitter ist eine Stadt südlich von Braunschweig.', 'Die Polizei hat gesagt:', 'Auf der A7 war ein Unfall.', 'Bei dem Unfall wurde kein Mensch verletzt.', 'Aber bei dem Unfall ist ein Lkw umgekippt.', 'Jetzt liegt der Lkw auf der A7.', 'Und er blockiert alle Fahr·spuren in Richtung Norden.', 'Deshalb haben wir den Teil von der A7 gesperrt.', 'Und deshalb ist ein langer Stau.', 'Die Polizei hat auch gesagt:', 'Jetzt müssen Fach·leute den Lkw von der A7 holen.', 'Das dauert noch einige Stunden.', 'Deshalb bleibt der Teil von der A7 wahrscheinlich noch einige Stunden gesperrt.', 'Noch weiß kein Mensch:', 'Warum ist der Lkw umgekippt?'], 'Niedersachsen: Mann stirbt bei Unfall': ['Am Donnerstag·morgen war ein Unfall.', 'Der Unfall war in Samern.', 'Samern ist ein Ort im Süd·westen von Niedersa

In [ ]:
headlines = soup.find_all("h2")
for headline in headlines:
    #print(headline.prettify(), end="\n"*2)
    links = headline.find_all("a")  
    for link in links:
        print(link.prettify())
    

In [7]:
result = soup.find(id="abschriften-urkunden-ausweise-paesse")
print(result.prettify())

<h2 class="SP-Headline--section" id="abschriften-urkunden-ausweise-paesse">
 Abschriften &amp; Urkunden, Ausweise &amp; Pässe
</h2>



In [ ]:
category_elements = soup.find_all("section", class_="SP-Section")

for category_element in category_elements:
    print(category_element.prettify(), end="\n"*2)

In [43]:
links = []
easyLink = "?sp%3Aout=easy"
for category_element in category_elements:
    link_elements = category_element.find_all("a", class_="SP-Link SP-Iconized--left")
    for link_element in link_elements:
        if easyLink in link_element["href"]:
            links.append("https://stuttgart.de"+link_element["href"])
        
        
print(links)

['https://stuttgart.de/organigramm/leistungen/personalausweis-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/personalausweis-wegen-namenaenderung-beantragen-heirat-oder-scheidung-.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/reisepass-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/reisepass-beantragen-vorlaeufig.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/schwerbehinderten-ausweis-beantragen.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/urkunden-und-beglaubigte-abschriften-geburt-heirat-lebens-partnerschaft-und-sterbefall.php?sp%3Aout=easy', 'https://stuttgart.de/buergerinnen-und-buerger/menschen-mit-behinderung/die-beauftragte-fuer-menschen-mit-behinderung.php?sp%3Aout=easy', 'https://stuttgart.de/service/behoerdennummer-115/index.php?sp%3Aout=easy', 'https://stuttgart.de/buergerinnen-und-buerger/menschen-mit-behind

In [23]:
df = pd.DataFrame(columns=['URL', 'Titel', 'Text_Leicht', 'Text_Allgemein'])

In [ ]:
i = 1
for url in links:
    try:
        url2 = url.replace("?sp%3Aout=easy", "")
        page = requests.get(url)
        page2 = requests.get(url2)

        soup = BeautifulSoup(page.content, "html.parser")
        soup2 = BeautifulSoup(page2.content, "html.parser")

        result = soup.find("h1", class_="SP-Headline--article")
        title = result.text

        print(title, url)

        result = soup.find("div", class_="SP-ArticleContent")
        for div in result.find_all("section", class_="SP-Text"):
            for h2 in div.find_all("h2", id="kontakt"):
                div.decompose()

        text = result.getText(separator=u' ')

        result2 = soup2.find("div", class_="SP-ArticleContent")

        for div in result2.find_all("section", class_="SP-Section SP-Grid__full"):
            for h2 in div.find_all("h2"):
                if(h2.text == 'Ihr Kontakt zu uns:'):
                    div.decompose()

        for div in result2.find_all("section", class_="SP-Text"):
            for h2 in div.find_all("h2", id="service-telefon"):
                div.decompose()    

        text2 = result2.getText(separator=u' ')    

        df_article = pd.DataFrame({
            'URL': url,
            'Titel': title,
            'Text_Leicht': text,
            'Text_Allgemein': text2},
            index=[i]
        )

        i += 1
        df = pd.concat([df, df_article])
    except:
        print("Error with ", title, url)

In [74]:
df.to_csv("../../02_Data/ndr.csv")